In [1]:
import csv
import json
from dotenv import load_dotenv
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute

In [2]:
load_dotenv(verbose=True)

True

In [3]:
class SingleTableModel(Model):
    class Meta:
        table_name = "florentia_academy_db"
        region = 'us-west-2' 
    pk = UnicodeAttribute(hash_key=True)
    sk = UnicodeAttribute(range_key=True)
    data = UnicodeAttribute(null=True)
    
    @classmethod
    def teacher_pk(cls, id):
        return f"TEACHER#{id}"

    @classmethod
    def student_pk(cls, id):
        return f"STUDENT#{id}"

    @classmethod
    def course_pk(cls, id):
        return f"COURSE#{id}"

    @classmethod
    def semester_pk(cls, id):
        return f"SEMESTER#{id}"
        
    @classmethod
    def course_sk(cls):
        return f"METADATA"
        
    @classmethod
    def teacher_sk(cls):
        return f"METADATA"

    @classmethod
    def course_sk(cls):
        return f"METADATA"

    @classmethod
    def student_sk(cls):
        return f"METADATA"

    @classmethod
    def semester_sk(cls):
        return f"METADATA"


In [4]:
def get_reader(csv_filename):
    with open(csv_filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        return list(reader)
    
def empty_table():
    for item in SingleTableModel.scan():
        item.delete()

def seed_courses():
    reader = get_reader("courses.csv")
    for row in reader:
        course_id, course_name = row
        course = SingleTableModel(pk=SingleTableModel.course_pk(course_id), sk=SingleTableModel.course_sk())
        data = {
            "course_name": course_name
        }
        course.data = json.dumps(data)
        course.save()

def seed_students():
    reader = get_reader("students.csv")
    for row in reader:
        id, last, first, dob = row
        student = SingleTableModel(pk=SingleTableModel.student_pk(id), sk=SingleTableModel.student_sk())
        data = {
            "first": first,
            "last": last,
            "dob": dob
        }
        student.data = json.dumps(data)
        student.save()

def seed_teachers():
    reader = get_reader("teachers.csv")
    for row in reader:
        id, last, first, dob, degree = row
        teacher = SingleTableModel(pk=SingleTableModel.teacher_pk(id), sk=SingleTableModel.teacher_sk())
        data = {
            "first": first,
            "last": last,
            "dob": dob,
            "degree": degree
        }
        teacher.data = json.dumps(data)
        teacher.save()
        
def seed_semester():
    semester1 = SingleTableModel(pk=SingleTableModel.semester_pk("2023S1"), sk=SingleTableModel.semester_sk())
    data = {}
    data["description"] = "2023 Semester 1"
    semester1.data = json.dumps(data)
    semester1.save()

def seed():
    empty_table()
    seed_semester()
    seed_courses()
    seed_students()
    seed_teachers()

In [5]:
seed()